In [1]:
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import roc_auc_score
from keras.layers import Input
from keras.models import Model

Using TensorFlow backend.


In [0]:
import os
import gc
gc.collect()
os.environ['KAGGLE_USERNAME'] = "afsan123"
os.environ['KAGGLE_KEY'] = "dcaf5555702cabce34cb49b78c852642"

In [3]:
!kaggle competitions download -c state-farm-distracted-driver-detection

  0% 0.00/193k [00:00<?, ?B/s]
100% 193k/193k [00:00<00:00, 74.8MB/s]
100% 3.99G/4.00G [02:21<00:00, 33.5MB/s]
100% 4.00G/4.00G [02:22<00:00, 30.2MB/s]
  0% 0.00/92.9k [00:00<?, ?B/s]
100% 92.9k/92.9k [00:00<00:00, 94.5MB/s]


In [4]:
!7z x /content/driver_imgs_list.csv.zip


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/                   1 file, 95118 bytes (93 KiB)

Extracting archive: /content/driver_imgs_list.csv.zip
--
Path = /content/driver_imgs_list.csv.zip
Type = zip
Physical Size = 95118

  0%    Everything is Ok

Size:       491359
Compressed: 95118


In [5]:
!7z x /content/imgs.zip


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/                   1 file, 4292669227 bytes (4094 MiB)

Extracting archive: /content/imgs.zip
  4% 4096 Open              --
Path = /content/imgs.zip
Type = zip
Physical Size = 4292669227
64-bit = +

  0%      0% 433 - test/img_100496.jpg                                0% 771 - test/img_100880.jpg                                1% 1046 - test/img_10120.jpg                                1% 1299 - test/img_101488.jpg                               

In [0]:
#data augmentation 
#increasing 1000 images in each class
import os
import random
from skimage import io
import random
from scipy import ndarray
import skimage as sk
from skimage import transform
from skimage import util
from skimage.util import random_noise
from skimage.transform import rotate
import cv2
folder_path = '/content/train/c9'

num_files_desired = 1000

images = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
num_generated_files = 0
image_to_transform=[]
while num_generated_files <= num_files_desired:
    
    image_path = random.choice(images)
    x= cv2.imread(image_path)
    image_to_transform.append(x)
    num_generated_files=num_generated_files+1
    #print(num_generated_files)

In [0]:
#print(type(image_to_transform))
img=np.array(image_to_transform)

In [0]:
import cv2
folder_path='/content/train/c9'
i=0
#new_file_path = '%s/augmented_image_%i.jpg' % (folder_path, num_generated_files)
#print(new_file_path)
# write image to the disk
x=32000
for i in range(1001):
    new_file_path = '%s/img_%s.jpg' % (folder_path,x)
    x=x+1
    sk.io.imsave(new_file_path, img[i])

In [0]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.4, 
    zoom_range = 0.3,
    horizontal_flip = True,
    validation_split=0.2) 

In [10]:
train_generator = train_datagen.flow_from_directory(
    '/content/train',
    target_size = (32, 32),
    color_mode = 'rgb',
    batch_size = 32,
    class_mode='categorical',
    subset='training'
    )
validation_generator = train_datagen.flow_from_directory(
    '/content/train',
    target_size = (32, 32),
    color_mode = 'rgb',
    batch_size = 32,
    class_mode = 'categorical',
    subset='validation')

Found 18727 images belonging to 10 classes.
Found 4677 images belonging to 10 classes.


In [0]:
import cv2
import matplotlib.pyplot as plt
batch_size=32
from keras.callbacks import History 
history = History()

In [0]:
#VGG16
batch_size=32
num_val_samples=22424
model = Sequential()
model.add(Conv2D(32, (3,3), input_shape = (32, 32, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(32, (3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(64, (3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64,activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy',
              optimizer = 'rmsprop',
              metrics = ['accuracy'])

history=model.fit_generator(train_generator, samples_per_epoch=25764, nb_epoch=10, 
                    validation_data=validation_generator, nb_val_samples=800)






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., validation_data=<keras_pre..., steps_per_epoch=805, epochs=10, validation_steps=800)`





Epoch 1/10





805/805 [==============================] - 258s 321ms/step - loss: 2.0024 - acc: 0.2548 - val_loss: 1.5661 - val_acc: 0.4476
Epoch 2/10
805/805 [==============================] - 252s 313ms/step - loss: 1.3912 - acc: 0.4896 - val_loss: 1.0362 - val_acc: 0.6470
Epoch 3/10
805/805 [==============================] - 254s 315ms/step - loss: 1.0577 - acc: 0.6236 - val_loss: 0.8235 - val_acc: 0.7139
Epoch 4/10
805/805 [==============================] - 258s 320ms/step - loss: 0.8513 - acc: 0.7013 - val_loss: 0.5983 - val_acc: 0.8068
Epoch 5/10
805/805 [==============================] - 259s 321ms/step - loss: 0.7260 - acc: 0.7482 - val_loss: 0.6223 - val_acc: 0.7843
Epoch 6/10
805/805 [==============================] - 258s 321ms/step - loss: 0.6336 - acc: 0.7838 - val_loss: 0.4572 - val_acc: 0.8465
Epoch 7/10
805/805 [==============================] - 258s 320ms/step - loss: 0.5845 - acc: 0.8030 - val_loss: 0.4070 - val_acc: 0.8677
Epoch 8/10
805/805 [====================

# dumping the model for further use

# loading the model for future use

In [0]:
import matplotlib.pyplot as plt
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, color='red', label='Training loss')
plt.plot(epochs, val_loss, color='green', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [0]:
acc = history.history['acc']
val_acc = history.history['val_acc']
plt.plot(epochs, acc, color='red', label='Training acc')
plt.plot(epochs, val_acc, color='green', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [0]:
model.save('model.h5')
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model.h5")
print("Saved model to disk")

In [0]:
from keras.models import model_from_json
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

In [0]:
os.listdir('/content/train/c0')

In [0]:
from keras.preprocessing import image
import numpy as np
from keras.models import Model
from keras.models import load_model
layer_outputs = [layer.output for layer in loaded_model.layers]
activation_model = Model(inputs=loaded_model.input, outputs=layer_outputs)
img51 =image.load_img('/content/train/c0/img_97738.jpg',target_size = (32,32))
img5 = image.img_to_array(img51)
activations = activation_model.predict(img5.reshape(1,32,32,3))
print(activations)
 
def display_activation(activations, col_size, row_size, act_index): 
    activation = activations[act_index]
    activation_index=0
    fig, ax = plt.subplots(row_size, col_size, figsize=(row_size*2.5,col_size*1.5))
    for row in range(0,row_size):
        for col in range(0,col_size):
            ax[row][col].imshow(activation[0, :, :, activation_index], cmap='gray')
            activation_index += 1

In [0]:
display_activation(activations, 8, 4, 0)

In [0]:
loss= model.evaluate_generator(validation_generator,steps=800)
print("Loss: " + str(loss[0]) + "     Accuracy" + str(loss[1]))

In [0]:
from keras.preprocessing import image
import numpy as np
from keras.models import load_model
from keras.models import Model
test_image =image.load_img('/content/train/c0/img_97738.jpg',target_size = (32,32))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image,axis=0)
classes =loaded_model.predict(test_image)
print (classes)

In [0]:
train_generator1 = train_datagen.flow_from_directory(
    '/content/train',
    target_size = (77, 77),
    color_mode = 'rgb',
    batch_size = 32,
    class_mode='categorical',
    subset='training'
    )
validation_generator1 = train_datagen.flow_from_directory(
    '/content/train',
    target_size = (77, 77),
    color_mode = 'rgb',
    batch_size = 32,
    class_mode = 'categorical',
    subset='validation')

In [0]:
#ANN
model1 = Sequential()
model1.add(Dense(64, activation='relu', input_shape=(32,32,3)))
model1.add(Dropout(0.5))
model1.add(Flatten())
model1.add(Dense(64, activation='relu'))
model1.add(Dropout(0.5))
model1.add(Dense(10, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model1.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

history=model1.fit_generator(train_generator, samples_per_epoch=25764, nb_epoch=10, 
                    validation_data=validation_generator, validation_steps=800)

In [0]:
model1.save('model1.h5')
model1_json = model1.to_json()
with open("model1.json", "w") as json_file:
    json_file.write(model1_json)
model1.save_weights("model1.h5")
print("Saved model to disk")

In [0]:
from keras.models import model_from_json
json_file = open('model1.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model1 = model_from_json(loaded_model_json)
loaded_model1.load_weights("model1.h5")
print("Loaded model from disk")

In [0]:
from keras.preprocessing import image
import numpy as np
from keras.models import Model
from keras.models import load_model
layer_outputs = [layer.output for layer in loaded_model1.layers]
activation_model = Model(inputs=loaded_model1.input, outputs=layer_outputs)
img51 =image.load_img('/content/train/c0/img_97738.jpg',target_size = (32,32))
img5 = image.img_to_array(img51)
a = activation_model.predict(img5.reshape(1,32,32,3))
print(activations)
 
def display_activation(activations, col_size, row_size, act_index): 
    activation = activations[act_index]
    activation_index=0
    fig, ax = plt.subplots(row_size, col_size, figsize=(row_size*2.5,col_size*1.5))
    for row in range(0,row_size):
        for col in range(0,col_size):
            ax[row][col].imshow(activation[0, :, :, activation_index], cmap='gray')
            activation_index += 1

In [0]:
display_activation(activations, 8, 4, 1)

In [0]:
import matplotlib.pyplot as plt
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, color='red', label='Training loss')
plt.plot(epochs, val_loss, color='green', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [0]:
acc = history.history['acc']
val_acc = history.history['val_acc']
plt.plot(epochs, acc, color='red', label='Training acc')
plt.plot(epochs, val_acc, color='green', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [0]:
from keras.preprocessing import image
import numpy as np
from keras.models import load_model
from keras.models import Model
test_image =image.load_img('/content/train/c0/img_97738.jpg',target_size = (32,32))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image,axis=0)
classes =loaded_model1.predict(test_image)
print (classes)

In [0]:
#Inception
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.4,
    zoom_range = 0.3,
    horizontal_flip = True,
    validation_split=0.2) 

test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator3 = train_datagen.flow_from_directory(
    '/content/train',
    target_size = (32, 32),
    color_mode = 'rgb',
    batch_size = 32,
    class_mode='categorical',
    subset='training'
    )
validation_generator3 = train_datagen.flow_from_directory(
      '/content/train',
    target_size = (32, 32),
    color_mode = 'rgb',
    batch_size = 32,
    class_mode = 'categorical',
    subset='validation')

input1 = Input(shape = (32, 32, 3))

layer_1 = Conv2D(64, (1,1), padding='same', activation='relu')(input1)
layer_1 = Conv2D(64, (3,3), padding='same', activation='relu')(layer_1)
layer_2 = Conv2D(64, (1,1), padding='same', activation='relu')(input1)
layer_2 = Conv2D(64, (5,5), padding='same', activation='relu')(layer_2)
layer_3 = MaxPooling2D((3,3), strides=(1,1), padding='same')(input1)
layer_3 = Conv2D(64, (1,1), padding='same', activation='relu')(layer_3)

output_layer1 = keras.layers.concatenate([layer_1, layer_2, layer_3], axis = 3)
layer_4 = Conv2D(64, (1,1), padding='same', activation='relu')(output_layer1)
layer_4 = Conv2D(64, (3,3), padding='same', activation='relu')(layer_4)
layer_5 = Conv2D(64, (1,1), padding='same', activation='relu')(output_layer1)
layer_5 = Conv2D(64, (3,3), padding='same', activation='relu')(layer_5)
layer_5 = Conv2D(64, (3,3), padding='same', activation='relu')(layer_5)
layer_6 = MaxPooling2D((3,3), strides=(1,1), padding='same')(output_layer1)
layer_6 = Conv2D(64, (1,1), padding='same', activation='relu')(layer_6)

output_layer2 = keras.layers.concatenate([layer_4, layer_5, layer_6], axis = 3)
output = Flatten()(output_layer2)
out    = Dense(10, activation='softmax')(output)

model2 = Model(inputs = input1, outputs = out)

from keras.optimizers import SGD
epochs = 10
alpha = 0.01
decay = alpha/epochs
sgd = SGD(lr=alpha, momentum=0.9, decay=decay, nesterov=False)
model2.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
history=model2.fit_generator(train_generator3, samples_per_epoch=25764, nb_epoch=epochs, 
                    validation_data=validation_generator3, nb_val_samples=800)

In [0]:
model2.save('model2.h5')
model2_json = model2.to_json()
with open("model2.json", "w") as json_file:
    json_file.write(model2_json)
model2.save_weights("model2.h5")
print("Saved model to disk")

In [0]:
from keras.models import model_from_json
json_file = open('model2.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model2 = model_from_json(loaded_model_json)
loaded_model2.load_weights("model2.h5")
print("Loaded model from disk")

In [0]:
img_path = '/content/train/c0/img_97738.jpg'
img = image.load_img(img_path, target_size=(32, 32))
img_tensor = image.img_to_array(img)
img_tensor = np.expand_dims(img_tensor, axis=0)
img_tensor /= 255.

In [0]:
from keras import models
layer_outputs = [layer.output for layer in loaded_model2.layers[1:12]] 
# Extracts the outputs of the top 12 layers
activation_model = models.Model(inputs=loaded_model2.input, outputs=layer_outputs)

activations = activation_model.predict(img_tensor) 
first_layer_activation = activations[0]

In [0]:
layer_names = []
for layer in loaded_model2.layers[:12]:
    layer_names.append(layer.name) 
    
images_per_row = 16

for layer_name, layer_activation in zip(layer_names, activations): 
    n_features = layer_activation.shape[-1] 
    size = layer_activation.shape[1] 
    n_cols = n_features // images_per_row 
    display_grid = np.zeros((size * n_cols, images_per_row * size))
    for col in range(n_cols): 
        for row in range(images_per_row):
            channel_image = layer_activation[0,
                                             :, :,
                                             col * images_per_row + row]
            channel_image -= channel_image.mean() 
            channel_image /= channel_image.std()
            channel_image *= 64
            channel_image += 128
            channel_image = np.clip(channel_image, 0, 255).astype('uint8')
            display_grid[col * size : (col + 1) * size, # Displays the grid
                         row * size : (row + 1) * size] = channel_image
    scale = 1. / size
    plt.figure(figsize=(scale * display_grid.shape[1],
                        scale * display_grid.shape[0]))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='gray')

In [0]:
import matplotlib.pyplot as plt
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, color='red', label='Training loss')
plt.plot(epochs, val_loss, color='green', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [0]:
acc = history.history['acc']
val_acc = history.history['val_acc']
plt.plot(epochs, acc, color='red', label='Training acc')
plt.plot(epochs, val_acc, color='green', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [0]:
from keras.preprocessing import image
import numpy as np
from keras.models import load_model
from keras.models import Model
test_image =image.load_img('/content/train/c0/img_97738.jpg',target_size = (32,32))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image,axis=0)
classes =loaded_model2.predict(test_image)
print (classes)

In [0]:
from keras.optimizers import SGD
epochs=10
alpha = 0.01
decay = alpha/epochs
sgd = SGD(lr=alpha, momentum=0.9, decay=decay, nesterov=False)
loaded_model2.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
loss=loaded_model2.evaluate_generator(validation_generator3,steps=800)
print("Loss: " + str(loss[0]) + "     Accuracy " + str(loss[1]))

In [0]:
#Alexnet
shape = (77,77,3)

classifier = Sequential()

classifier.add(Conv2D(96,(5,5),strides = (2,2),input_shape=shape,activation = 'relu'))
classifier.add(MaxPooling2D(pool_size=(2,2),strides =(2,2)))
classifier.add(Conv2D(256,(3,3),activation = 'relu'))
classifier.add(MaxPooling2D(pool_size=(2,2),strides =(2,2)))
classifier.add(Conv2D(384,3,3,activation = 'relu'))
classifier.add(Conv2D(384,3,3,activation = 'relu'))
classifier.add(Conv2D(384,3,3,activation = 'relu'))
classifier.add(MaxPooling2D(pool_size=(2,2),strides =(2,2)))

classifier.add(Flatten())
classifier.add(Dense(4096, input_shape=shape,activation = 'relu'))
classifier.add(Dropout(0.4))
classifier.add(Dense(4096, activation = 'relu'))
classifier.add(Dropout(0.4))
classifier.add(Dense(1000, activation = 'relu'))
classifier.add(Dropout(0.4))
classifier.add(Dense(10, activation = 'softmax'))
classifier.summary()

epochs = 10
alpha = 0.01
decay = alpha/epochs
sgd = SGD(lr=alpha, momentum=0.9, decay=decay, nesterov=True)
classifier.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
history=classifier.fit_generator(train_generator1, samples_per_epoch=25764, nb_epoch=epochs, 
                    validation_data=validation_generator1, nb_val_samples=800)

In [0]:
classifier.save('classifier.h5')
classifier_json = classifier.to_json()
with open("classifier.json", "w") as json_file:
    json_file.write(classifier_json)
classifier.save_weights("classifier.h5")
print("Saved model to disk")

In [0]:
from keras.models import model_from_json
json_file = open('classifier.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_classifier = model_from_json(loaded_model_json)
loaded_classifier.load_weights("classifier.h5")
print("Loaded model from disk")

In [0]:
from keras.preprocessing import image
import numpy as np
from keras.models import Model
from keras.models import load_model
layer_outputs = [layer.output for layer in loaded_classifier.layers]
activation_model = Model(inputs=loaded_classifier.input, outputs=layer_outputs)
img51 =image.load_img('/content/train/c0/img_97738.jpg',target_size = (77,77))
img5 = image.img_to_array(img51)
activations = activation_model.predict(img5.reshape(1,77,77,3))
print(activations)
 
def display_activation(activations, col_size, row_size, act_index): 
    activation = activations[act_index]
    activation_index=0
    fig, ax = plt.subplots(row_size, col_size, figsize=(row_size*2.5,col_size*1.5))
    for row in range(0,row_size):
        for col in range(0,col_size):
            ax[row][col].imshow(activation[0, :, :, activation_index], cmap='gray')
            activation_index += 1

In [0]:
display_activation(activations, 8, 4, 0)

In [0]:
import matplotlib.pyplot as plt
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, color='red', label='Training loss')
plt.plot(epochs, val_loss, color='green', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [0]:
acc = history.history['acc']
val_acc = history.history['val_acc']
plt.plot(epochs, acc, color='red', label='Training acc')
plt.plot(epochs, val_acc, color='green', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [0]:
from keras.optimizers import SGD
epochs=10
alpha = 0.01
decay = alpha/epochs
sgd = SGD(lr=alpha, momentum=0.9, decay=decay, nesterov=True)
loaded_classifier.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
loss=loaded_classifier.evaluate_generator(validation_generator1,steps=800)
print("Loss: " + str(loss[0]) + "     Accuracy " + str(loss[1]))

In [0]:
from keras.preprocessing import image
import numpy as np
from keras.models import load_model
from keras.models import Model
test_image =image.load_img('/content/train/c0/img_97738.jpg',target_size = (77,77))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image,axis=0)
classes =loaded_classifier.predict(test_image)
print (classes)

In [0]:
#Inception
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.4,
    zoom_range = 0.3,
    horizontal_flip = True,
    validation_split=0.2) 

test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator3 = train_datagen.flow_from_directory(
    '/content/train',
    target_size = (32, 32),
    color_mode = 'rgb',
    batch_size = 32,
    class_mode='categorical',
    subset='training'
    )
validation_generator3 = train_datagen.flow_from_directory(
      '/content/train',
    target_size = (32, 32),
    color_mode = 'rgb',
    batch_size = 32,
    class_mode = 'categorical',
    subset='validation')

input1 = Input(shape = (32, 32, 3))

layer_1 = Conv2D(64, (1,1), padding='same', activation='relu')(input1)
layer_1 = Conv2D(64, (3,3), padding='same', activation='relu')(layer_1)
layer_2 = Conv2D(64, (1,1), padding='same', activation='relu')(input1)
layer_2 = Conv2D(64, (5,5), padding='same', activation='relu')(layer_2)
layer_3 = MaxPooling2D((3,3), strides=(1,1), padding='same')(input1)
layer_3 = Conv2D(64, (1,1), padding='same', activation='relu')(layer_3)

output = keras.layers.concatenate([layer_1, layer_2, layer_3], axis = 3)

output = Flatten()(output)
out    = Dense(10, activation='softmax')(output)

model2 = Model(inputs = input1, outputs = out)

from keras.optimizers import SGD
epochs = 10
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history=model2.fit_generator(train_generator3, samples_per_epoch=25764, nb_epoch=epochs, 
                    validation_data=validation_generator3, nb_val_samples=800)

In [0]:
import matplotlib.pyplot as plt
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, color='red', label='Training loss')
plt.plot(epochs, val_loss, color='green', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [0]:
acc = history.history['acc']
val_acc = history.history['val_acc']
plt.plot(epochs, acc, color='red', label='Training acc')
plt.plot(epochs, val_acc, color='green', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [0]:
train_generator4 = train_datagen.flow_from_directory(
    '/content/train',
    target_size = (32, 32),
    color_mode = 'grayscale',
    batch_size = 32,
    class_mode='categorical',
    subset='training'
    )
validation_generator4 = train_datagen.flow_from_directory(
    '/content/train',
    target_size = (32, 32),
    color_mode = 'grayscale',
    batch_size = 32,
    class_mode = 'categorical',
    subset='validation')

In [0]:
plt.matshow(first_layer_activation[0, :, :, 4], cmap='viridis')

In [0]:
import matplotlib.pyplot as plt
img_path = '/content/train/c0/img_97738.jpg'
img = image.load_img(img_path, target_size=(77, 77))
img_tensor = image.img_to_array(img)
img_tensor = np.expand_dims(img_tensor, axis=0)
img_tensor /= 255.
plt.imshow(img_tensor[0])
plt.show()

In [0]:
layer_names = []
for layer in loaded_classifier.layers[:6]:
    layer_names.append(layer.name) 
    
images_per_row = 16

for layer_name, layer_activation in zip(layer_names, activations): 
    n_features = layer_activation.shape[-1] 
    size = layer_activation.shape[1] 
    n_cols = n_features // images_per_row 
    display_grid = np.zeros((size * n_cols, images_per_row * size))
    for col in range(n_cols): 
        for row in range(images_per_row):
            channel_image = layer_activation[0,
                                             :, :,
                                             col * images_per_row + row]
            channel_image -= channel_image.mean() 
            channel_image /= channel_image.std()
            channel_image *= 64
            channel_image += 128
            channel_image = np.clip(channel_image, 0, 255).astype('uint8')
            display_grid[col * size : (col + 1) * size, 
                         row * size : (row + 1) * size] = channel_image
    scale = 1. / size
    plt.figure(figsize=(scale * display_grid.shape[1],
                        scale * display_grid.shape[0]))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='gray')

In [0]:
shape = (77,77,3)

classifier1 = Sequential()

classifier1.add(Conv2D(96,(5,5),strides = (2,2),input_shape=shape,activation = 'relu'))
classifier1.add(MaxPooling2D(pool_size=(2,2),strides =(2,2)))
classifier1.add(Conv2D(384,3,3,activation = 'relu'))
classifier1.add(Conv2D(384,3,3,activation = 'relu'))
classifier1.add(Conv2D(384,3,3,activation = 'relu'))
classifier1.add(MaxPooling2D(pool_size=(2,2),strides =(2,2)))

classifier1.add(Flatten())
classifier1.add(Dense(1024, input_shape=shape,activation = 'relu'))
classifier1.add(Dropout(0.4))
classifier1.add(Dense(1024, activation = 'relu'))
classifier1.add(Dropout(0.4))
classifier1.add(Dense(100, activation = 'relu'))
classifier1.add(Dropout(0.4))
classifier1.add(Dense(10, activation = 'softmax'))
classifier1.summary()

epochs = 10
alpha = 0.01
decay = alpha/epochs
sgd = SGD(lr=alpha, momentum=0.9, decay=decay, nesterov=True)
classifier1.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
history=classifier1.fit_generator(train_generator1, samples_per_epoch=25764, nb_epoch=epochs, 
                    validation_data=validation_generator1, nb_val_samples=800)